# Air BNB Seattle Dataset Project by Carina Breu
## Introduction


## Research Questions

## Dataset Overview

In [ ]:
#Import statements
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

% matplotlib inline

## Data Preparation

## Modeling

## Evaluation

## Deployment